# Street sign with Pre-Trained WideResNet

With additional shield net 

In [5]:
from torch.utils.data import Dataset, DataLoader
from os.path import join
import pandas as pd 
from PIL import Image
from torch.optim import SGD
import seaborn as sb 
from gtsrb import GTSRB
from detectors import EnsembleDetector, SemanticDetector, LogicOnlyDetector
import os 
from pytorch_ood.utils import fix_random_seed
import seaborn as sb 
import torch 

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-GPU-bb1ccb6e-2bc9-c7a1-b25d-3eef9033e192/0/0" 

sb.set()

device="cuda:0"
root = "data/"


fix_random_seed()

def seed_worker(worker_id):
    fix_random_seed()

g = torch.Generator()
g.manual_seed(0)


In [6]:
import torchvision
from torchvision.transforms import ToTensor, Resize, Compose
import torch 
from torch.utils.data import DataLoader

trans = Compose([ToTensor(), Resize((32, 32))])

train_data = GTSRB(root=root, train=True, transforms=trans)
test_data = GTSRB(root=root, train=False, transforms=trans)

In [7]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2, worker_init_fn=seed_worker)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2, worker_init_fn=seed_worker)

In [8]:
from torch import nn
from torchvision.models.resnet import resnet18
from pytorch_ood.model import WideResNet
from functools import partial

def myfeatures(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = self.avgpool(x)
    x = torch.flatten(x, 1)
    return x
    
# def override 
def Model(num_classes=None, *args, **kwargs):
    model = resnet18(num_classes=num_classes)
    model.myfeatures = partial(myfeatures, model)
#     model = WideResNet(*args, num_classes=1000, **kwargs, pretrained="imagenet32")
#     model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model


In [9]:
from tqdm.notebook import tqdm 
import numpy as np 

def train_model(att_index, num_classes):
    """
    train a model for the given attribute index 
    """
    trans = Compose([ToTensor(), Resize((32, 32))])
    train_data = GTSRB(root=root, train=True, transforms=trans)
    test_data = GTSRB(root=root, train=False, transforms=trans)
    
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2, worker_init_fn=seed_worker)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2, worker_init_fn=seed_worker)
    
    model = Model(num_classes=num_classes).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True)

    accs = []

    for epoch in range(20):
        running_loss = 0.0
        model.train()
        bar = tqdm(train_loader)
        for inputs, y in bar:
            labels = y[:, att_index]
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss = 0.8 * running_loss + 0.2 * loss.item()
            bar.set_postfix({"loss": running_loss})

        correct = 0
        total = 0

        with torch.no_grad():
            model.eval()

            for inputs, y in test_loader:
                labels = y[:, att_index]
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, dim=1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Accuracy of the network on the test images: {correct / total:.2%}')

    return model 

# Sign Network 

In [10]:
from torch.utils.data import DataLoader
from pytorch_ood.utils import is_known
from tqdm.notebook import tqdm 
from pytorch_ood.dataset.img import TinyImages300k
from pytorch_ood.utils import ToUnknown
from torch.utils.data import random_split

def train_sign_model():
    tiny = TinyImages300k(root=root, download=True, transform=trans, target_transform=ToUnknown())
    data_train_out, data_test_out, _ = random_split(tiny, [50000, 10000, 240000], generator=torch.Generator().manual_seed(123))

    train_data_noatt = GTSRB(root=root, train=True, transforms=trans, target_transform=lambda y: y[0])
    test_data_noatt = GTSRB(root=root, train=False, transforms=trans, target_transform=lambda y: y[0])

    new_loader = DataLoader(train_data_noatt + data_train_out, batch_size=32, shuffle=True, num_workers=10, worker_init_fn=seed_worker)
    new_test_loader = DataLoader(test_data_noatt + data_test_out, batch_size=32, shuffle=False, num_workers=10, worker_init_fn=seed_worker)

    model = Model(num_classes=2).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True)

    accs = []

    for epoch in range(20):
        running_loss = 0.0
        model.train()
        
        bar = tqdm(new_loader)
        for inputs, y in bar:
            labels = is_known(y).long()
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss = 0.8 * running_loss + 0.2 * loss.item()
            bar.set_postfix({"loss": running_loss})

        correct = 0
        total = 0

        with torch.no_grad():
            model.eval()

            for inputs, y in new_test_loader:
                labels = is_known(y).long()
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Accuracy of the shape network on the test images: {correct / total:.2%}')
        accs.append(correct / total)

    return model 

# OOD Evaluation 

In [14]:
from pytorch_ood.dataset.img import (LSUNCrop, LSUNResize, Textures, TinyImageNetCrop, TinyImageNetResize)
from pytorch_ood.detector import EnergyBased, MaxSoftmax, Mahalanobis, MaxLogit, ViM, Entropy
from pytorch_ood.utils import ToRGB, OODMetrics


def evaluate(label_net, shape_net, color_net, shield_net):
    _ = label_net.eval()
    _ = shape_net.eval()
    _ = color_net.eval()
    _ = shield_net.eval()
    
    results = []

    trans = Compose([Resize(size=(32, 32)), ToRGB(), ToTensor()])
    data_in = GTSRB(root=root, train=False, transforms=trans, target_transform=lambda y: y[0])
    data_in_train = GTSRB(root=root, train=True, transforms=trans, target_transform=lambda y: y[0])
    loader_train = DataLoader(data_in_train, batch_size=1024, shuffle=False, worker_init_fn=seed_worker)
    # dataset_out_test = Textures(root=root, transform=trans, target_transform=ToUnknown(), download=True)
    
    detectors = {
        "MSP": MaxSoftmax(label_net),
        "Energy": EnergyBased(label_net),
        "Mahalanobis": Mahalanobis(label_net.myfeatures),
        "MaxLogit": MaxLogit(label_net),
        "Entropy": Entropy(label_net),
        "ViM": ViM(label_net.myfeatures, w=label_net.fc.weight, b=label_net.fc.bias, d=64),
        "Logic": LogicOnlyDetector(
                label_net, 
                shape_net, 
                color_net, 
                GTSRB(root=root).class_to_shape, 
                GTSRB(root=root).class_to_color, 
        ),
        "Logic+": LogicOnlyDetector(
                label_net, 
                shape_net, 
                color_net, 
                GTSRB(root=root).class_to_shape, 
                GTSRB(root=root).class_to_color, 
                sign_net=shield_net
        ),
        "Semantic-OE": SemanticDetector(
            label_net, 
            shape_net, 
            color_net, 
            GTSRB(root=root).class_to_shape, 
            GTSRB(root=root).class_to_color, 
            sign_net=shield_net
        ),
        "Semantic": SemanticDetector(
                label_net, 
                shape_net, 
                color_net, 
                GTSRB(root=root).class_to_shape, 
                GTSRB(root=root).class_to_color, 
        ),
        "Ensemble": EnsembleDetector(label_net, shape_net, color_net)
    }
    
    for detector_name, detector in detectors.items():
        print(f"Fitting {detector_name}")
        try:
            detector.fit(loader_train, device=device)
        except TypeError:
            detector.fit(loader_train)
        
    datasets = {d.__name__: d for d in (LSUNCrop, LSUNResize, Textures, TinyImageNetCrop, TinyImageNetResize)}
    
    for detector_name, detector in detectors.items():
        for data_name, dataset_c in datasets.items():
            data_out = dataset_c(root=root, transform=trans, target_transform=ToUnknown(), download=True)
            loader = DataLoader(data_in+data_out, batch_size=1024, shuffle=False, worker_init_fn=seed_worker)
            
            scores = []
            ys = []
            
            with torch.no_grad():
                for x, y in loader:
                    scores.append(detector(x.to(device)))
                    ys.append(y.to(device))
                    
                scores = torch.cat(scores, dim=0).cpu()
                ys = torch.cat(ys, dim=0).cpu()
            
            metrics = OODMetrics()
            metrics.update(scores, ys)
            r = metrics.compute()
            r.update({
                "Method": detector_name,
                "Dataset": data_name
            })
            print(r)
            results.append(r)
    
    return results 

In [15]:
def evaluate_acc(net, att_idx=0, oe=False):
    _ = net.eval()
    
    if oe:
        target_trans = lambda y: torch.tensor(1)
    else:
         target_trans = lambda y: y[att_idx]

    trans = Compose([Resize(size=(32, 32)), ToRGB(), ToTensor()])
    data_in = GTSRB(root=root, train=False, transforms=trans, target_transform=target_trans)
    loader = DataLoader(data_in, batch_size=1024, shuffle=False, worker_init_fn=seed_worker)
            
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            predicted = outputs.max(dim=1).indices
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return correct / total  

def evaluate_accs(label_net, shape_net, color_net, shield_net):
    r = {}
    names = ("Label", "Color", "Shape",)
    
    for n, net in enumerate((label_net, color_net, shape_net)): 
        acc = evaluate_acc(net, n)
        r[names[n]] = acc
    
    acc = evaluate_acc(shield_net, oe=True)
    r["Sign"] = acc
    
    return [r] 

In [16]:
results = []
results_acc = []

for trial in range(10):
    shield_net = train_sign_model()
    shape_net = train_model(att_index=2, num_classes=5)
    color_net = train_model(att_index=1, num_classes=4)
    label_net = train_model(att_index=0, num_classes=43)
    
    res = evaluate(label_net, shape_net, color_net, shield_net)
    res_acc = evaluate_accs(label_net, shape_net, color_net, shield_net)
    
    for r in res:
        r.update({"Seed": trial})
        
    for r in res_acc:
        r.update({"Seed": trial})
    
    results += res
    results_acc += res_acc

RuntimeError: No CUDA GPUs are available

In [ ]:
import pandas as pd 
result_df = pd.DataFrame(results)
tab = (result_df.groupby(by="Method").agg(["mean", "sem"]) * 100)[["AUROC", "AUPR-IN", "AUPR-OUT", "FPR95TPR"]].sort_values(by=("AUROC", "mean"))
string = tab.to_latex(float_format="%.2f").replace("& 0", "& $\pm$ 0").replace("& 1", "& $\pm$ 1")
print(string)

In [ ]:
(result_df.groupby(by=["Method", "Seed"]).mean() * 100).groupby("Method").agg(["mean", "sem"])

In [ ]:
from scipy.stats import ttest_ind

for metric in ["AUROC", "AUPR-IN", "AUPR-OUT", "FPR95TPR"]:
    sem_auroc = np.array(result_df[result_df["Method"] == "Semantic"].groupby(by=["Method", "Seed"]).mean(numeric_only=True)[metric])
    sem_ensemble =  np.array(result_df[result_df["Method"] == "Ensemble"].groupby(by=["Method", "Seed"]).mean(numeric_only=True)[metric])
    stat, p = ttest_ind(sem_auroc, sem_ensemble, equal_var=False) # , permutations=100000
    print(f"Metric: {metric} -> {p}")

In [ ]:
# dataset gives np.array([label, color, shape])
print((pd.DataFrame(results_acc) * 100).agg(["mean", "sem"]).to_latex(float_format="%.2f"))
# results_acc = evaluate_accs(label_net, shape_net, color_net, shield_net)
# print(results_acc)

# Eval

In [ ]:
msp = MaxSoftmax(label_net)
semantic_detector = SemanticDetector(
                label_net, 
                shape_net, 
                color_net, 
                GTSRB(root=root).class_to_shape, 
                GTSRB(root=root).class_to_color)

In [ ]:
from pytorch_ood.utils import TensorBuffer

data_in = GTSRB(root=root, train=False, transforms=trans, target_transform=lambda y: y[0])
data_out = Textures(root=root, transform=trans, target_transform=ToUnknown(), download=True)

loader = DataLoader(data_in + data_out, shuffle=False, batch_size=1024)

buffer = TensorBuffer()

for x, y in loader:
    scores_msp = msp(x.to(device))
    buffer.append("msp", scores_msp)
    
    scores_sem = semantic_detector(x.to(device))
    buffer.append("sem", scores_sem)
    
    buffer.append("y", y)

In [ ]:
import matplotlib.pyplot as plt 
sb.set_style("white")

df = pd.DataFrame({"Scores": buffer["sem"].numpy() + 1, "label": buffer["y"]})
df["Type"] = df["label"].apply(lambda x: "IN" if is_known(x) else "OOD")

fig, ax = plt.subplots(figsize=(4,3))

sb.kdeplot(data=df, x="Scores", clip=[0,1], common_norm=False, fill=True, hue="Type", palette="crest", bw_method=0.2) # , palette="crest" # , 
sb.move_legend(ax, "upper right")
plt.xlabel("Normalized Outlier Score")
sb.despine(left=True, right=True, top=True)
plt.xlim([0,1])
plt.yticks([])
plt.tight_layout()
plt.savefig("img/scores-gtsrb.pdf", bbox_inches="tight")

In [ ]:
sb.set_style("white")

df = pd.DataFrame({"Scores": buffer["msp"].numpy() + 1, "label": buffer["y"]})
df["Type"] = df["label"].apply(lambda x: "IN" if is_known(x) else "OOD")

fig, ax = plt.subplots(figsize=(4,3))

sb.kdeplot(data=df, x="Scores", clip=[0,1], common_norm=False, fill=True, hue="Type", palette="crest", bw_method=0.2) # , palette="crest" # , 
plt.xlim([0,1])
sb.move_legend(ax, "upper right")
plt.xlabel("Normalized Outlier Score")
sb.despine(left=True, right=True, top=True)

plt.yticks([])
plt.tight_layout()